In [1]:
import pandas as pd
import numpy as np
import folium
import requests
import json
import pprint
import datetime as dt
import re

In [2]:
room_data = pd.read_csv('room.csv',encoding='CP949')
room_data.head()

,건물명(상호명),건물위치,주택유형구분,세대수(객실수),건축연도
0,NaN,전라북도 전주시 덕진구 고랑동 819-5,다세대주택(3세대),3,19920810
1,NaN,전라북도 전주시 덕진구 금암동 1569-6,다세대주택(3세대),6,19891206
2,NaN,전라북도 전주시 덕진구 금암동 1588-23,다세대주택(3세대),3,19890803
3,휴엔하임 주상복합 아파트,전라북도 전주시 덕진구 금암동 1639-1,"공동주택(아파트), 업무시설(오피스텔), 근린생활시설",30,20161128
4,NaN,전라북도 전주시 덕진구 금암동 525-83,다세대주택,4,19891121


In [3]:
room_data.head()

,건물명(상호명),건물위치,주택유형구분,세대수(객실수),건축연도
0,NaN,전라북도 전주시 덕진구 고랑동 819-5,다세대주택(3세대),3,19920810
1,NaN,전라북도 전주시 덕진구 금암동 1569-6,다세대주택(3세대),6,19891206
2,NaN,전라북도 전주시 덕진구 금암동 1588-23,다세대주택(3세대),3,19890803
3,휴엔하임 주상복합 아파트,전라북도 전주시 덕진구 금암동 1639-1,"공동주택(아파트), 업무시설(오피스텔), 근린생활시설",30,20161128
4,NaN,전라북도 전주시 덕진구 금암동 525-83,다세대주택,4,19891121


In [4]:
room_data.describe()

,세대수(객실수)
count,8734.000000
mean,8.135791
std,11.724445
min,0.000000
25%,3.000000
50%,8.000000
75%,11.000000
max,826.000000


In [5]:
## TODO
## 건물 위치를 참조하여 folium에 좌표 찍기
## 건축 연도를 참조하여 room_data에 신축 / 준신축 / 보통 / 재건축필요 등의 등급으로 분류

# room_data['건물위치']
# key = "7F8A26C5-7C0A-318B-A521-FE1AA35942DB"
# arr = []
# arr = []
# f = open("./loc.txt", 'w')
# f.close()
# import time
# for i in room_data['건물위치']:
#     f = open("./loc.txt", 'a')
#     location = i.strip()
#     response = requests.get(f"http://api.vworld.kr/req/address?service=address&request=getCoord&key={key}&address={location}&type=PARCEL")
#     if 'result' in response.json()['response']:
#         x= float(response.json()['response']['result']['point']['x'])
#         y = float(response.json()['response']['result']['point']['y'])
#         arr.append([float(x),float(y)])
#         f.write(str(x))
#         f.write(',')
#         f.write(str(y))
#         f.write('\n')
#         f.close()
#     else:
#         print(location, response.json())
#     time.sleep(0.25)

In [6]:
f = open('./loc.txt', 'r')
lines = f.readlines()
map1 = folium.Map(location=[35.83001,127.125001], zoom_start=13)
arr = []
for line in lines:
    x,y = map(float, line.split(','))
    x = round(x,2)
    y = round(y,2)
    if [x,y] not in arr:
        arr.append([x,y])

for content in arr:
    x,y = content
    folium.Marker([y, x], popup='원룸', icon=folium.Icon(color='red',icon='info-sign')).add_to(map1)
    folium.CircleMarker([y, x], radius=30, color='#3186cc', fill_color='#3186cc', popup='원룸').add_to(map1)
map1.save('map1.html')

In [7]:
p = re.compile(r'\d+')
room_data['건축연도'] = room_data['건축연도'].map(lambda x: x if p.search(x) else '0')


this_year = dt.datetime.today().year
score = 0
if this_year-room_data['건축연도'].map(lambda x: int(x[:4]))<=5:
    score = 5
elif this_year-room_data['건축연도'].map(lambda x: int(x[:4]))>=6 and this_year-room_data['건축연도'].map(lambda x: int(x[:4]))<=10:
    score = 4
elif this_year-room_data['건축연도'].map(lambda x: int(x[:4]))>=11 and this_year-room_data['건축연도'].map(lambda x: int(x[:4]))<=15:
    score = 3
elif this_year-room_data['건축연도'].map(lambda x: int(x[:4]))>=16 and this_year-room_data['건축연도'].map(lambda x: int(x[:4]))<=20:
    score = 2
elif this_year-room_data['건축연도'].map(lambda x: int(x[:4]))>=21 and this_year-room_data['건축연도'].map(lambda x: int(x[:4]))<=100:
    score = 1
else :
    score = 0

room_data['construct_point'] = score
room_data.head()

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().